# [LAB 05] Data Preprocessing 3 / Feature Engineering

### 오늘 pdf에 나오는 모든 표들은 다 중요한거임!!!! 외워외워 

## 4. 명목형(범주형) 변수 인코딩

In [ ]:
# 데이터분석과정:
# 수집 -> 전처리 -> 피쳐엔지니어링 -> EDA -> 확증적 데이터분석 -> 머신러닝 (모델링)
    # 전통적인 분석 과정: 
        # 수집 -> 전처리 -> 피쳐엔지니어링 -> EDA -> 확증적 데이터분석 -> 머신러닝
    # 최근 분석 트렌드: 
        # 수집 -> 전처리 -> 피쳐엔지니어링 -> EDA -> 머신러닝
    # 1차 플젝: 수집 -> 전처리 -> 피쳐엔지니어링 -> EDA -> 확증적 데이터분석
    # (2차 플젝): Dashboard 만들기
    # 3차 플젝: 수집 -> 전처리 -> 피쳐엔지니어링 -> EDA -> 머신러닝

# 피쳐엔지니어링 ( ↓ 1) ~ 3): 변수 늘리기) - 머신러닝기법 살짝 들어감
    # 1) 파생변수
    # 2) 범주형 인코딩
    # 3) 연속형 스케일링 (표준화, 정규화)
    # for 차원의 저주 해결 (변수 줄이기, 차원 축소): 
        # 주성분 분석(PCA) or Feature(변수) Importance
        # Feature Selection

# EDA (탐색적 데이터 분석)
    # 목적: 현상탐구 하는 것
    # 기술통계, 시각화

# 확증적 데이터분석
    # 목적: 현상에 대한 원인 찾기
    # 추론통계, 분류분석
    # 변수 선택: (목적에 따라)
        # 수치형 => 예측모형 (for 선형회귀)
            # ex) 집값, 선호도
        # 명목형 => 분류모형 (for 로지스틱회귀)
            # ex) 주로 의학에서 쓰임: 죽었냐살았냐? 양성이냐음성이냐?

# 머신러닝
    # a.k.a. 모델링 
    # 목적: 예측
    # 알고리즘 종류: 개~ 마늠!
        # 선형 계열: Linear Regression, Rogistic Regression, Lidge, Lasso,...
        # 트리 계열: Decision Tree, Randon Forest, Support Vector Machine,...
        # 이런거 싹~ 다 해보고 그 중 제일 정확도 높은 알고리즘을 선택해야 됨
        # 요즘 트렌드: 
            # Vagging: 위 나열한 종류들 중 n개 이상을 결합
            # Boosting: 1개의 알고리즘을 여러번 학습시키는 방법
            # 앙상블 = Vagging + Boosting 
            # 요즘은: 
                # 1. 범주형 변수 갯수 만큼 희소행렬(OneHotEncoding)여부 다 지정해보고 
                # 2. 모든 ML 모델들에 지정한 희소행렬여부로 다 ML 돌려보고
                # 3. 그 중 정확도 높은 상위 5개 모델 선택해서 
                # 4. 이 5개의 모델을 앙상블 시켜야 된다!

# 확증적 데이터 분석까지 한 이후 결과 만드는 것: Dash board
    # Dash board 목적: 웹 개발

In [ ]:
# 인코딩: 변수를 0 또는 1로 변환하는 것
    # 주로 문자열로 구성된 명목형(범주형) 변수를 대상으로 함
    # after 인코딩 처리: dummy변수라고 지칭함
    # 범주형 변수 수가 많을 때: 희소행렬 문제
    # -> 해결방법:
        # 그룹화   
        # 상위 n개 범주형 변수들만 유지
        # Hassing Trick: 
            # ex) grade A, B, C, D, E 
            #  -> grouping to fewer groups as 상위성적(A, B), 하위성적(C, D, E)

# 명목형 변수 처리과정은 필수!!! 필수!!! 그래서 이번 단원 개 중요함!

### #01. 준비작업

In [ ]:
# 1. 패키지 참조하기:
from hossam import load_data
from pandas import DataFrame, get_dummies, merge # for 추론통계(get_dummies)
from sklearn.preprocessing import OneHotEncoder  # for 머신러닝

In [4]:
# 2. 샘플 데이터 가져오기:
origin = load_data('nursing_grades')
origin

[data] https://data.hossam.kr/data/lab05/nursing_grades.xlsx
[desc] 어느 간호학과 대학원에 지원한 학생들에 대한 합격/불합격 여부를 조사한 가상의 데이터(메타데이터 없음)
[!] Cannot read metadata


,이름,성별,필기점수,학부성적,병원경력,합격여부
접수코드,,,,,,
NRS0001,장은우,남,380,3.61,3,불합격
NRS0002,최지호,남,660,3.67,3,합격
NRS0003,김하준,남,800,4.00,1,합격
NRS0004,임아윤,여,640,3.19,4,합격
NRS0005,강하준,남,520,2.93,4,불합격
...,...,...,...,...,...,...
NRS0396,박지유,여,620,4.00,2,불합격
NRS0397,조하은,여,560,3.04,3,불합격
NRS0398,박하윤,여,460,2.63,2,불합격


In [6]:
# 3. 범주형 타입 변환:
# 중요: 항상 명목형 변수들은 '타입변환' 꼭 해주자!!! pdf에 생략 되있더라도!!!
df = origin.astype(
    {
        '성별': 'category',
        '병원경력': 'category',
        '합격여부': 'category'
    }
)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 400 entries, NRS0001 to NRS0400
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   이름      400 non-null    object  
 1   성별      400 non-null    category
 2   필기점수    400 non-null    int64   
 3   학부성적    400 non-null    float64 
 4   병원경력    400 non-null    category
 5   합격여부    400 non-null    category
dtypes: category(3), float64(1), int64(1), object(1)
memory usage: 14.1+ KB


### #02. Pandas 사용법 (일반 통계 모형에 추천)

In [19]:
# 1. 처리하고자 하는 범주형 변수 필드를 더미변수로 변환 (N개 생성): get_dummies()
# - 원본 데이터프레임과 더미변수로 변환할 컬럼 이름을 파라미터로 전달한다.
# - 원본 데이터프레임에서 기존의 컬럼은 제거되고 더미변수로 변경된 컬럼들이 
#   추가된 형태의 새로운 데이터프레임이 리턴된다.
# - 'dtype = "int"' 파라미터를 명시하지 않을 경우 'bool' 형식(True, False)
#   으로 값이 생성되므로 항상 이 옵션 쓰기는 필수다!

df1 = get_dummies(df, columns = ['성별'], dtype = "int")
df1.head()  # 행 겁나 많은게 디폴트니까 .head() 쓰는거 습관 들이기

,이름,필기점수,학부성적,병원경력,합격여부,성별_남,성별_여
접수코드,,,,,,,
NRS0001,장은우,380,3.61,3,불합격,1,0
NRS0002,최지호,660,3.67,3,합격,1,0
NRS0003,김하준,800,4.00,1,합격,1,0
NRS0004,임아윤,640,3.19,4,합격,0,1
NRS0005,강하준,520,2.93,4,불합격,1,0


In [10]:
# 2. N-1개의 더미변수 생성하기: 'drop_first = True (기본값 = False)'
#----------------------강사님 설명-------------------------------
# drop_first = True 옵션:
    # 이유: 더미변수 생성은 범주형 변수를 인코딩(0 또는 1)으로 바꾸는 것이기 때문에
    # 데이터 하나가 0이면 다른 하나는 1인건 자동임
    # -> 궅이 더미변수를 다 나타낼 필요 x -> n-1개
#----------------------------------------------------------------
df2 = get_dummies(
    df, columns = ['성별'], dtype = "int", drop_first = True
    )
df2.head()

,이름,필기점수,학부성적,병원경력,합격여부,성별_여
접수코드,,,,,,
NRS0001,장은우,380,3.61,3,불합격,0
NRS0002,최지호,660,3.67,3,합격,0
NRS0003,김하준,800,4.00,1,합격,0
NRS0004,임아윤,640,3.19,4,합격,1
NRS0005,강하준,520,2.93,4,불합격,0


In [13]:
# 두개 이상의 명목형 변수 처리방법: 
    # columns 안에 리스트 형태로 처리할 명목형 변수들 다 나열
df3 = get_dummies(
    df, columns = ['성별', '병원경력'], dtype = "int", drop_first = True
    )
df3.head()

,이름,필기점수,학부성적,합격여부,성별_여,병원경력_2,병원경력_3,병원경력_4
접수코드,,,,,,,,
NRS0001,장은우,380,3.61,불합격,0,0,1,0
NRS0002,최지호,660,3.67,합격,0,0,1,0
NRS0003,김하준,800,4.00,합격,0,0,0,0
NRS0004,임아윤,640,3.19,합격,1,0,0,1
NRS0005,강하준,520,2.93,불합격,0,0,0,1


### #03. Scikit-Learn 사용방법 (트리 기반 머신러닝 모형에 추천)

In [ ]:
#----------------------강사님 설명-------------------------------
# df['변수']: 필드 하나 (Series 열)가져욤
# df[['변수']]: 이차원 가져옴

# 아래 설명은 강사님이 두번 이상 강조하신 부분!!:
    # 알고리즘 당 어떤 경우에 true(1) 값이 들어가는지, 
    # false(0) 값이 들어가는지는 본인 각자가 다 정리해놔야됨!!
#----------------------------------------------------------------

In [14]:
# 1. 1단계 - 방법1): 명목형 데이터의 차원 변환
    # 1) 이유: Sklearn의 OneHotEncoder는 
    #    '2차원 배열'이나 'DataFrame 객체'를 파라미터로 요구하기 때문에 
    #    인코딩 처리할 명목형 변수를 
    #    차원 변환 시키거나 (for 2차원 배열)
    #    데이터프레임의 형태를 유지한 상태로 (for DataFrame 객체)
    #    입력값을 생성해야 한다

# 방법1) 행 수에 관계 없이 1개의 열을 갑는 2차원 배열로 변환:
X = df['성별'].values.reshape(-1, 1)
X

[['남'], ['남'], ['남'], ['여'], ['남'], ..., ['여'], ['여'], ['여'], ['여'], ['여']]
Length: 400
Categories (2, object): ['남', '여']

In [ ]:
# 2. 1단계 - 방법2): 데이터프레임의 형식을 유지하는 변수 추출

# 방법2) 데이터 프레임 형태를 유지하면서 열을 추출하는 경우
# (filter 함수도 동일)
X = df[['성별']]
X.head()

,성별
접수코드,
NRS0001,남
NRS0002,남
NRS0003,남
NRS0004,여
NRS0005,남


In [ ]:
# 3. 2단계: OneHotEncoding 처리:
encoder = OneHotEncoder(
    sparse_output = False, drop = None
    )# ↑: 희소행렬 여부 지정, ↑: 첫번째항목 삭제여부 
result = encoder.fit_transform(X)#(↑종류: None, "first")
result

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.

In [17]:
# 4. 3단계: 인코딩 결과를 데이터 프레임으로 생성:

# 1) 컬럼명 생성: encoder.get_feature_names_out(['열이름'])
new_cols = encoder.get_feature_names_out(['성별'])
new_cols

array(['성별_남', '성별_여'], dtype=object)

In [18]:
# 2) DataFrame으로 병합:
one_hot_df = DataFrame(
    result, columns = new_cols, index = df.index
)
one_hot_df.head()

,성별_남,성별_여
접수코드,,
NRS0001,1.0,0.0
NRS0002,1.0,0.0
NRS0003,1.0,0.0
NRS0004,0.0,1.0
NRS0005,1.0,0.0


In [20]:
# 5. 4단계: 원본 데이터프레임과 병합하기:
df_copy = df.copy()
df4 = merge(
    df_copy, one_hot_df, left_index = True, right_index = True
)
df4.head()

,이름,성별,필기점수,학부성적,병원경력,합격여부,성별_남,성별_여
접수코드,,,,,,,,
NRS0001,장은우,남,380,3.61,3,불합격,1.0,0.0
NRS0002,최지호,남,660,3.67,3,합격,1.0,0.0
NRS0003,김하준,남,800,4.00,1,합격,1.0,0.0
NRS0004,임아윤,여,640,3.19,4,합격,0.0,1.0
NRS0005,강하준,남,520,2.93,4,불합격,1.0,0.0
